https://cloud.google.com/bigquery-ml/docs/create-machine-learning-model

# Create machine learning models in BigQuery ML

This tutorial introduces users to BigQuery ML using the Google Cloud console.

BigQuery ML enables users to create and execute machine learning models in BigQuery by using SQL queries. The goal is to democratize machine learning by enabling SQL practitioners to build models using their existing tools and to increase development speed by eliminating the need for data movement.

In this tutorial, you use the sample [Google Analytics sample dataset for BigQuery](https://support.google.com/analytics/answer/7586738?hl=en&ref_topic=3416089) to create a model that predicts whether a website visitor will make a transaction. For information on the schema of the Analytics dataset, see [BigQuery export schema](https://support.google.com/analytics/answer/3437719) in the Google Analytics Help Center.

## Objectives

In this tutorial, you use:

* BigQuery ML to create a binary logistic regression model using the `CREATE MODEL` statement
* The `ML.EVALUATE` function to evaluate the ML model
* The `ML.PREDICT` function to make predictions using the ML model

## Costs

This tutorial uses billable components of Google Cloud, including the following:

* BigQuery
* BigQuery ML
For more information on BigQuery costs, see the [BigQuery pricing](https://cloud.google.com/bigquery/pricing) page.

For more information on BigQuery ML costs, see the [BigQuery ML pricing](https://cloud.google.com/bigquery-ml/pricing) page.

In [1]:
import os
from google.cloud import bigquery

In [2]:
shell_output = !gcloud config get-value project
PROJECT_ID = shell_output[0]
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
client = bigquery.Client()
%load_ext google.cloud.bigquery

## Create your dadaset

The first step is to create a BigQuery dataset to store your ML model. To create your dataset:

In [3]:
dataset = client.create_dataset("bqml_tutorial", exists_ok=True)

## Create your model

Next, you create a logistic regression model using the Google Analytics sample dataset for BigQuery. The following standard SQL query is used to create the model you use to predict whether a website visitor will make a transaction.

```sql
CREATE OR REPLACE MODEL `bqml_tutorial.sample_model`
OPTIONS(model_type='logistic_reg') AS
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20160801' AND '20170630'
```

In addition to creating the model, running a query that contains the `CREATE MODEL` statement trains the model using the data retrieved by your query's `SELECT` statement.

### Query details

The [CREATE MODEL](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create) clause is used to create and train the model named `bqml_tutorial.sample_model`.

The `OPTIONS(model_type='logistic_reg')` clause indicates that you are creating a [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression) model. A logistic regression model tries to split input data into two classes and gives the probability the data is in one of the classes. Usually, what you are trying to detect (such as whether an email is spam) is represented by 1 and everything else is represented by 0. If the logistic regression model outputs 0.9, there is a 90% probability the input is what you are trying to detect (the email is spam).

This query's `SELECT` statement retrieves the following columns that are used by the model to predict the probability a customer will complete a transaction:

* `totals.transactions` — The total number of ecommerce transactions within the session. If the number of transactions is `NULL`, the value in the `label` column is set to `0`. Otherwise, it is set to `1`. These values represent the possible outcomes. Creating an alias named `label` is an alternative to setting the `input_label_cols=` option in the `CREATE MODEL` statement.
* `device.operatingSystem` — The operating system of the visitor's device.
* `device.isMobile` — Indicates whether the visitor's device is a mobile device.
* `geoNetwork.country` — The country from which the sessions originated, based on the IP address.
* `totals.pageviews` — The total number of page views within the session.

The `FROM` clause — `bigquery-public-data.google_analytics_sample.ga_sessions_*` — indicates that you are querying the Google Analytics sample dataset. This dataset is in the `bigquery-public-data` project. You are querying a set of tables sharded by date. This is represented by the wildcard in the table name: `google_analytics_sample.ga_sessions_*`.

The `WHERE` clause — `_TABLE_SUFFIX BETWEEN '20160801' AND '20170630'` — limits the number of tables scanned by the query. The date range scanned is August 1, 2016 to June 30, 2017.

### Run the CREATE MODEL query

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `bqml_tutorial.sample_model`
OPTIONS(model_type='logistic_reg') AS
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20160801' AND '20170630'

The query takes several minutes to complete. After the first iteration is complete, your model (`sample_model`) appears in the navigation panel. Because the query uses a `CREATE MODEL` statement to create a model, you do not see query results.

You can observe the model as it's being trained by viewing the **Model stats** tab. As soon as the first iteration completes, the tab is updated. The stats continue to update as each iteration completes.

## Get training statistics

To see the results of the model training, you can use the [ML.TRAINING_INFO](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-train) function, or you can view the statistics in the Google Cloud console.

Machine learning is about creating a model that can use data to make a prediction. The model is essentially a function that takes inputs and applies calculations to the inputs to produce an output — a prediction.

Machine learning algorithms work by taking several examples where the prediction is already known (such as the historical data of user purchases) and iteratively adjusting various weights in the model so that the model's predictions match the true values. It does this by minimizing how wrong the model is using a metric called loss.

The expectation is that for each iteration, the loss should be decreasing (ideally to zero). A loss of zero means the model is 100% accurate.

To see the model training statistics that were generated when you ran the `CREATE MODEL` query:

In [4]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_tutorial.sample_model`)

Query is running:   0%|          |

Downloading:   0%|          |

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,8,0.043878,0.045445,25.6,10986
1,0,7,0.044654,0.045499,25.6,52261
2,0,6,0.047345,0.048273,12.8,10853
3,0,5,0.053888,0.053337,6.4,13371
4,0,4,0.067776,0.066406,3.2,11522
5,0,3,0.097545,0.096203,1.6,11059
6,0,2,0.169802,0.168849,0.8,12415
7,0,1,0.320692,0.320174,0.4,12399
8,0,0,0.521573,0.521380,0.2,9606


The **Training Data Loss** column represents the loss metric calculated after the given iteration on the training dataset. Since you performed a logistic regression, this column is the [log loss](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression). The **Evaluation Data Loss** column is the same loss metric calculated on the holdout dataset (data that is held back from training to validate the model).

BigQuery ML automatically splits your input data into a training set and a holdout set to avoid [overfitting](https://en.wikipedia.org/wiki/Overfitting) the model. This is necessary so that the training algorithm doesn't so closely tailor to the known data that it doesn't generalize to unseen, new examples.

Training Data Loss and Evaluation Data Loss are average loss values, averaged over all examples in the respective sets.

For more details on the `ML.TRAINING_INFO` function, see the [BigQuery ML Syntax Reference](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-train).

## Evaluate your model

After creating your model, you evaluate the performance of the classifier using the `ML.EVALUATE` function. The `ML.EVALUATE` function evaluates the predicted values against the actual data. You can also use the [ML.ROC_CURVE](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-roc) function for logistic regression specific metrics.

In this tutorial you are using a binary classification model that detects transactions. The two classes are the values in the `label` column: `0` (no transactions) and `1` (transaction made).

The query used to evaluate the model is as follows:

```sql
SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_tutorial.sample_model`, (
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))
```

### Query details

The top-most `SELECT` statement retrieves the columns from your model.

The `FROM` clause uses the [ML.EVALUATE](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate) function against your model: `bqml_tutorial.sample_model`.

This query's nested `SELECT` statement and `FROM` clause are the same as those in the `CREATE MODEL` query.

The `WHERE` clause — `_TABLE_SUFFIX BETWEEN '20170701' AND '20170801'` — limits the number of tables scanned by the query. The date range scanned is July 1, 2017 to August 1, 2017. This is the data you're using to evaluate the predictive performance of the model. It was collected in the month immediately following the time period spanned by the training data.

### Run the ML.EVALUATE query

In [5]:
%%bigquery
SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_tutorial.sample_model`, (
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))

Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.468504,0.110801,0.985343,0.179217,0.046242,0.98175


Because you performed a logistic regression, the results include the following columns:

* [precision](https://developers.google.com/machine-learning/glossary/#precision) — A metric for classification models. Precision identifies the frequency with which a model was correct when predicting the positive class.
* [recall](https://developers.google.com/machine-learning/glossary/#recall) — A metric for classification models that answers the following question: Out of all the possible positive labels, how many did the model correctly identify?
* [accuracy](https://developers.google.com/machine-learning/glossary/#accuracy) — Accuracy is the fraction of predictions that a classification model got right.
* [f1_score](https://en.wikipedia.org/wiki/F1_score) — A measure of the accuracy of the model. The f1 score is the harmonic average of the precision and recall. An f1 score's best value is 1. The worst value is 0.
* [log_loss](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression) — The loss function used in a logistic regression. This is the measure of how far the model's predictions are from the correct labels.
* [roc_auc](https://developers.google.com/machine-learning/glossary/#AUC) — The area under the [ROC](https://developers.google.com/machine-learning/glossary/#ROC) curve. This is the probability that a classifier is more confident that a randomly chosen positive example is actually positive than that a randomly chosen negative example is positive. For more information, see [Classification](https://developers.google.com/machine-learning/crash-course/classification/video-lecture) in the Machine Learning Crash Course.

## Use your model to predict outcomes

Now that you have evaluated your model, the next step is to use it to predict an outcome. You use your model to predict the number of transactions made by website visitors from each country.

The query used to predict the outcome is as follows:

```sql
SELECT
  country,
  SUM(predicted_label) as total_predicted_purchases
FROM
  ML.PREDICT(MODEL `bqml_tutorial.sample_model`, (
SELECT
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(totals.pageviews, 0) AS pageviews,
  IFNULL(geoNetwork.country, "") AS country
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))
GROUP BY country
ORDER BY total_predicted_purchases DESC
LIMIT 10
```

### Query details

The top-most `SELECT` statement retrieves the `country` column and sums the `predicted_label` column. This column is generated by the `ML.PREDICT` function. When you use the `ML.PREDICT` function the output column name for the model is `predicted_<label_column_name>`. For linear regression models, `predicted_label` is the estimated value of `label`. For logistic regression models, `predicted_label` is the most likely label, which in this case is either `0` or `1`.

The [ML.PREDICT](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict) function is used to predict results using your model: `bqml_tutorial.sample_model`.

This query's nested `SELECT` statement and `FROM` clause are the same as those in the `CREATE MODEL` query.

The `WHERE` clause — `_TABLE_SUFFIX BETWEEN '20170701' AND '20170801'` — limits the number of tables scanned by the query. The date range scanned is July 1, 2017 to August 1, 2017. This is the data for which you're making predictions. It was collected in the month immediately following the time period spanned by the training data.

The `GROUP BY` and `ORDER BY` clauses group the results by country and order them by the sum of the predicted purchases in descending order.

The `LIMIT` clause is used here to display only the top 10 results.

### Run the ML.PREDICT query

In [6]:
%%bigquery
SELECT
  country,
  SUM(predicted_label) as total_predicted_purchases
FROM
  ML.PREDICT(MODEL `bqml_tutorial.sample_model`, (
SELECT
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(totals.pageviews, 0) AS pageviews,
  IFNULL(geoNetwork.country, "") AS country
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))
GROUP BY country
ORDER BY total_predicted_purchases DESC
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,country,total_predicted_purchases
0,United States,220
1,Taiwan,8
2,Canada,7
3,Turkey,2
4,Japan,2
5,India,2
6,Indonesia,1
7,Venezuela,1
8,Singapore,1
9,St. Lucia,1


## Predict purchases per user

In this example, you try to predict the number of transactions each website visitor will make. This query is identical to the previous query except for the `GROUP BY` clause. Here the `GROUP BY` clause — `GROUP BY fullVisitorId` — is used to group the results by visitor ID.

In [7]:
%%bigquery
SELECT
  fullVisitorId,
  SUM(predicted_label) as total_predicted_purchases
FROM
  ML.PREDICT(MODEL `bqml_tutorial.sample_model`, (
SELECT
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(totals.pageviews, 0) AS pageviews,
  IFNULL(geoNetwork.country, "") AS country,
  fullVisitorId
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))
GROUP BY fullVisitorId
ORDER BY total_predicted_purchases DESC
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,fullVisitorId,total_predicted_purchases
0,9417857471295131045,4
1,112288330928895942,2
2,057693500927581077,2
3,8388931032955052746,2
4,1280993661204347450,2
5,2158257269735455737,2
6,2969418676126258798,2
7,0376394056092189113,2
8,8064625150033508396,2
9,806992249032686650,2


## Clean up

To delete the resources created by this tutorial, execute the following code to delete the dataset and its contents:

In [8]:
client.delete_dataset(dataset, delete_contents=True)